<a href="https://colab.research.google.com/github/Tomawock/MLDM_COVID-19/blob/master/Province_protezione_civile.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [142]:
import pandas as pd
import numpy as np
import datetime as dt
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import os

Caricamento dataset provinciale dati covid e filtraggio sulla regione lombardia e le sue province e rimovione dei giorni nei quali non sono statia ggiornati i dati

In [143]:
#Carica dati dai file
protezionecivile = pd.read_csv("/content/dpc-covid19-ita-province.csv") 
isLombardia = protezionecivile["denominazione_regione"]=="Lombardia"
protezionecivile = protezionecivile[isLombardia]
print(protezionecivile.shape)
notEmpty= protezionecivile["totale_casi"]!= 0
protezionecivile = protezionecivile[notEmpty]
print(protezionecivile.shape)
nan_provincia=protezionecivile['sigla_provincia'].notnull()
protezionecivile = protezionecivile[nan_provincia]
print(protezionecivile.shape)
protezionecivile["data"] = pd.to_datetime(protezionecivile["data"], format='%Y-%m-%dT%H:%M:%S')
protezionecivile["data"] = protezionecivile["data"].dt.normalize()
protezionecivile

(3014, 11)
(2984, 11)
(2659, 11)


,data,stato,codice_regione,denominazione_regione,codice_provincia,denominazione_provincia,sigla_provincia,lat,long,totale_casi,note
176,2020-02-25,ITA,3,Lombardia,14,Sondrio,SO,46.170993,9.871475,1,NaN
177,2020-02-25,ITA,3,Lombardia,15,Milano,MI,45.466794,9.190347,8,NaN
178,2020-02-25,ITA,3,Lombardia,16,Bergamo,BG,45.694414,9.668425,18,NaN
180,2020-02-25,ITA,3,Lombardia,18,Pavia,PV,45.185093,9.160157,27,NaN
181,2020-02-25,ITA,3,Lombardia,19,Cremona,CR,45.133367,10.024209,53,NaN
...,...,...,...,...,...,...,...,...,...,...,...
30726,2020-10-04,ITA,3,Lombardia,19,Cremona,CR,45.133367,10.024209,6969,NaN
30727,2020-10-04,ITA,3,Lombardia,20,Mantova,MN,45.157268,10.792774,4248,NaN
30728,2020-10-04,ITA,3,Lombardia,97,Lecco,LC,45.855758,9.393392,3128,NaN
30729,2020-10-04,ITA,3,Lombardia,98,Lodi,LO,45.314407,9.503721,3815,NaN


In [144]:
print(protezionecivile.columns)
protezionecivile = protezionecivile.drop(columns=["stato", "codice_regione", "denominazione_regione","codice_provincia",'denominazione_provincia', "lat", "long", "note"])
print(protezionecivile.columns)

Index(['data', 'stato', 'codice_regione', 'denominazione_regione',
       'codice_provincia', 'denominazione_provincia', 'sigla_provincia', 'lat',
       'long', 'totale_casi', 'note'],
      dtype='object')
Index(['data', 'sigla_provincia', 'totale_casi'], dtype='object')


Calcolo la differenza tra i giorni per contare i nuovi positivi, ma il codice passando tra una provincia e 'altra sballa dato che i gionri poi si resettano

In [151]:
res=protezionecivile
protezionecivile = protezionecivile.sort_values(by=['sigla_provincia','data'])
res['nuovi_positivi'] = protezionecivile['totale_casi'].diff(periods=1)
res.drop(columns='totale_casi')

,data,sigla_provincia,nuovi_positivi
178,2020-02-25,BG,NaN
306,2020-02-26,BG,2.0
434,2020-02-27,BG,52.0
562,2020-02-28,BG,31.0
690,2020-02-29,BG,7.0
...,...,...,...
30123,2020-09-30,VA,17.0
30272,2020-10-01,VA,39.0
30421,2020-10-02,VA,30.0
30570,2020-10-03,VA,35.0


In [152]:
res.to_csv("protezioneCivile_province_nuovipositivi.csv", index=False)